In [ ]:
"""Feature Engineering for Scaled Weather Data (Direct Upload).ipynb

Automatically generated by Colab.
"""

import pandas as pd
import numpy as np
import os
# No need to import google.colab.drive as we are not mounting Drive
from google.colab import files # For downloading files

# IMPORTANT: If you are directly uploading your files to the Colab session,
# no need to mount Google Drive. The files will be accessible directly.

# 📁 Input Directory: Set to empty string "" or "." if files are uploaded directly
# to the root of your Colab session.
input_data_dir = "" # <--- MODIFIED: Assumes files are in the current working directory

# 📂 Output Directory: Where the new _engineered.csv files will be saved
# This directory will be created within your temporary Colab session storage.
output_data_dir = "Engineered_Data_Output/"

# Create the output directory if it doesn't exist
if not os.path.exists(output_data_dir):
    os.makedirs(output_data_dir)
    print(f"Created output directory: {output_data_dir}")

# 📄 List of scaled city filenames to be engineered
# Corrected list based on your input, assuming underscores for 'hourly_weather'
# and correct extensions/formatting.
scaled_filenames = [
    "ahmedabad_hourly_weather_scaled.csv",
    "bengaluru_hourly_weather_scaled.csv",
    "bhopal_hourly_weather_scaled.csv",
    "bhubaneswar_hourly_weather_scaled.csv",
    "bikaner_hourly_weather_scaled.csv",
    "chandigarh_hourly_weather_scaled.csv",
    "chennai_hourly_weather_scaled.csv",
    "coimbatore_hourly_weather_scaled.csv",
    "dehradun_hourly_weather_scaled.csv",
    "guwahati_hourly_weather_scaled.csv",
    "hyderabad_hourly_weather_scaled.csv",
    "imphal_hourly_weather_scaled.csv",
    "jaipur_hourly_weather_scaled.csv",
    "jaisalmer_hourly_weather_scaled.csv",
    "kochi_hourly_weather_scaled.csv",
    "kolkata_hourly_weather_scaled.csv",
    "lucknow_hourly_weather_scaled.csv", # Corrected from |
    "mumbai_hourly_weather_scaled.csv",
    "mysuru_hourly_weather_scaled.csv",
    "nagpur_hourly_weather_scaled.csv",
    "nashik_hourly_weather_scaled.csv",
    "new_delhi_hourly_weather_scaled.csv",
    "ooty_hourly_weather_scaled.csv",
    "patna_hourly_weather_scaled.csv",
    "port_blair_hourly_weather_scaled.csv",
    "pune_hourly_weather_scaled.csv",
    "ranchi_hourly_weather_scaled.csv",
    "shillong_hourly_weather_scaled.csv",
    "shimla_hourly_weather_scaled.csv", # Corrected: space replaced with underscore
    "siliguri_hourly_weather_scaled.csv", # Corrected: removed 'x'
    "surat_hourly_weather_scaled.csv",
    "thiruvananthapuram_hourly_weather_scaled.csv", # Corrected: .cvs to .csv
    "vadodara_hourly_weather_scaled.csv",
    "amini_island_hourly_weather_scaled.csv",
    "visakhapatnam_hourly_weather_scaled.csv"
]

# --- Feature Engineering Function ---
def engineer_features(df):
    """
    Applies feature engineering steps to a scaled DataFrame.
    Assumes df has 'time' as index and columns like 'temp', 'rhum', 'wspd', 'prcp', 'pres'.
    """
    # Create a copy to avoid SettingWithCopyWarning
    df_engineered = df.copy()

    # 1. Lagged Features (Previous time steps)
    df_engineered['temp_lag1'] = df_engineered['temp'].shift(1)
    df_engineered['rhum_lag1'] = df_engineered['rhum'].shift(1)
    df_engineered['wspd_lag1'] = df_engineered['wspd'].shift(1)
    df_engineered['wspd_lag2'] = df_engineered['wspd'].shift(2)
    df_engineered['wspd_lag3'] = df_engineered['wspd'].shift(3)

    # 2. Rolling Window Features (Aggregations over a period)
    # Ensure min_periods is 1 to get values even at the beginning of the series
    df_engineered['temp_roll_mean_3h'] = df_engineered['temp'].rolling(window=3, min_periods=1).mean()
    df_engineered['prcp_roll_sum_6h'] = df_engineered['prcp'].rolling(window=6, min_periods=1).sum()

    # 3. Time-based Features (from the datetime index)
    df_engineered['hour'] = df_engineered.index.hour
    df_engineered['month'] = df_engineered.index.month
    df_engineered['weekday'] = df_engineered.index.weekday # Monday=0, Sunday=6

    # Handle NaNs created by lagging and rolling features at the beginning of the series
    # Using linear interpolation first, then ffill/bfill for any remaining at edges
    df_engineered = df_engineered.interpolate(method='linear')
    df_engineered = df_engineered.fillna(method='ffill').fillna(method='bfill')

    # Drop any rows that still have NaNs (should be minimal after fills)
    df_engineered.dropna(inplace=True)

    return df_engineered

# --- Process each scaled file ---
for filename in scaled_filenames:
    print(f"\n--- Processing {filename} for Feature Engineering ---")
    try:
        # Construct full path to input scaled CSV file
        # Since data_dir is "", this will simply be the filename
        input_path = os.path.join(input_data_dir, filename)
        df_scaled = pd.read_csv(input_path, parse_dates=['time'], index_col='time')

        # Ensure index is timezone-aware if it lost it during saving/loading
        # (It should be from previous step, but re-confirming here for robustness)
        if df_scaled.index.tz is None:
            # Assuming your scaled data was converted to Asia/Kolkata timezone
            df_scaled.index = df_scaled.index.tz_localize('Asia/Kolkata')

        print(f"Loaded scaled data for {filename}. Head:")
        print(df_scaled.head())

        # Perform feature engineering
        df_engineered = engineer_features(df_scaled)

        print(f"Engineered Data Sample for {filename}:")
        print(df_engineered.head())
        print(f"Engineered Data Columns: {df_engineered.columns.tolist()}")
        print(f"Engineered Data Shape: {df_engineered.shape}")

        # Construct output filename and path for engineered data
        output_name = filename.replace('_scaled.csv', '_engineered.csv')
        output_path = os.path.join(output_data_dir, output_name)

        # Save the engineered DataFrame to the temporary Colab environment
        df_engineered.to_csv(output_path)
        print(f"Successfully engineered and saved data for: {filename} to {output_path}")

        # Download the engineered file to your local machine (this will trigger a download in your browser)
        files.download(output_path)

    except FileNotFoundError:
        print(f"Error: Input file not found - {filename}. Please ensure this file is uploaded directly to your Colab session.")
    except KeyError as e:
        print(f"Error: Missing expected column in {filename} during engineering - {e}.")
        print("Ensure the scaled CSVs have the required columns for engineering (e.g., 'temp', 'rhum', 'wspd', 'prcp', 'pres').")
    except Exception as e:
        print(f"An unexpected error occurred while processing {filename}: {e}")

print("\n--- All specified scaled files processed for feature engineering. ---")

Created output directory: Engineered_Data_Output/

--- Processing ahmedabad_hourly_weather_scaled.csv for Feature Engineering ---
Loaded scaled data for ahmedabad_hourly_weather_scaled.csv. Head:
                            temp      dwpt  rhum  prcp      wspd      pres
time                                                                      
2021-01-01 05:30:00+05:30  0.215  0.244582  0.54   0.0  0.045509  0.748649
2021-01-01 06:30:00+05:30  0.175  0.176471  0.51   0.0  0.056287  0.756757
2021-01-01 07:30:00+05:30  0.175  0.176471  0.51   0.0  0.056287  0.756757
2021-01-01 08:30:00+05:30  0.210  0.250774  0.56   0.0  0.045509  0.797297
2021-01-01 09:30:00+05:30  0.250  0.232198  0.45   0.0  0.067066  0.810811
Engineered Data Sample for ahmedabad_hourly_weather_scaled.csv:
                            temp      dwpt  rhum  prcp      wspd      pres  \
time                                                                         
2021-01-01 05:30:00+05:30  0.215  0.244582  0.54   0.0  0.0

<ipython-input-2-9772d4a5e8cf>:96: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_engineered = df_engineered.fillna(method='ffill').fillna(method='bfill')


Successfully engineered and saved data for: ahmedabad_hourly_weather_scaled.csv to Engineered_Data_Output/ahmedabad_hourly_weather_engineered.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<ipython-input-2-9772d4a5e8cf>:96: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_engineered = df_engineered.fillna(method='ffill').fillna(method='bfill')


Successfully engineered and saved data for: visakhapatnam_hourly_weather_scaled.csv to Engineered_Data_Output/visakhapatnam_hourly_weather_engineered.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


--- All specified scaled files processed for feature engineering. ---
